# Ma's Brown Dwarf Database
## Created by Emily Ma and Rebecca Jensen-Clem

This database currently contains information on 92 brown dwarfs compiled from various research papers during the summer of 2019. The objects listed in this database show strong evidence for measurements of polarization. These candidates have shown signs of asymmetry in previous research papers through measurements of low surface gravity, fast rotation periods, previous measurements of spectral variability, and other indicators.


### Setting Up the Database

The contents of the database can be pulled from https://github.com/emilyma53/brown-dwarfs. The 'brownDwarfCandidates.csv' contains information on the 92 objects. To query the database, the datascience module (created by the UC Berkeley Data Science Department) will need to be installed from https://github.com/data-8/datascience. Additionally, astroplan, astropy, and astroquery will need to be installed for extra functionionality in calculating the viewing constraints for each object.


*Note: Make sure the needed modules and files are in the current working directory, otherwise specify the path to the file.

bd = Table.read_table('/Example/path')

In [ ]:
from datascience import *
import numpy as np
bd = Table()
bd = Table.read_table('brownDwarfCandidates.csv')
bd


### How to Query the Database

The table is currently sorted with fourteen columns. 

Note: For gravity and variability measurements, view the papers listed in the 'gravity ref' and 'rotation ref' columns, respectively, for clarification on units. Not all cells in the table have measurements.

- To view an array of all of the column labels, use the command Table.labels

- To only view certain columns, use Table.select('label1', 'label2', etc.)

The command 'select' creates a new table with the selected columns.

<br>

In [ ]:
print(bd.labels)
bd.select('Name','Notes')

In [ ]:
bd_period = bd.select('Name', 'Per (hrs)', 'Rotation Ref')
bd_period

* You can also index into the table as normal

In [ ]:
bd[3][4]

### Filtering the Database

- You can filter the table using Table.where(column_label, predicate)

Some helpful predicates include:
* are.equal_to(x)
* are.above(x), are.above_or_equal_to(x)
* are.below(x), are.below_or_equal_to(x)
* are.between(x, y), are.between_or_equal_to(x, y)
* are.containing(s) — checks for strings that contain the substring s
* are.contained_in(s) — checks if the string s is contained in the string or column array


Run the cells below to see examples on how to use these predicates:

<br>

In [ ]:
bd.where('Name', are.equal_to('2MASS J00361617+1821104'))

In [ ]:
bd.where('Name', are.containing('J00361617'))

In [ ]:
bd1 = bd.where('Notes', are.containing('Radigan 2014'))
print(bd1.num_rows)
bd1.select('Name', 'Notes')

In [ ]:
bd2 = bd.where('Jmag', are.below(15))
bd2 = bd2.select('Name', 'Jmag')
bd2

### Sorting the Database
- To sort the table based on the given column values, use Table.sort(label, descending=false)

*change descending to True to sort in descending order

<br>

In [ ]:
bd2.sort('Jmag')

In [ ]:
bd.sort('Jmag', descending=True)

### Other Useful Functions

A few more useful functionalities of the datascience module are:

* Table.show(num_rows) — shows specified top num_rows of the table
* Table.to_df( ).to_csv(file_name, index = False) — saves the given table to the csv file name/path


For a more detailed documentation of the datascience module and other functions, see http://data8.org/datascience/index.html.



In [ ]:
bd.show(bd.num_rows)

In [ ]:
# bd.to_df().to_csv('brownDwarfCandidates.csv', index = False)

<br>

* Other useful functions that I have written myself are addRA(table) and addDEC(table), which adds any missing RA and DEC values in the form of '-63 25 05.595' and '16 48 15.6'. This requires the astroquery package as it queries the object in Simbad and uses the listed RA and DEC values. Can be helpful for filling in these values after adding more object names to the table.
* Numpy functions also work on each of the columns.

<br>

In [ ]:
from astroquery.simbad import Simbad
addRA(bd)
addDEC(bd)

In [ ]:
#**/ adds the Simbad RAJ2000 (h m s) to all of the rows in the first column of bd 
# if the row does not already have a value.
# 

def addRA(table):
    for i in range(table.num_rows):
        this_row = table.take(i)
        if this_row.column("RAJ2000")[0] == 'nan':
            ra = Simbad.query_object(this_row["Name"][0])
            ra = ra["RA"]
            if ra[0][0] == '+':
                table[1][i] = ra[0][1:]
            else:
                table[1][i] = ra[0]
                

#**/ adds the Simbad DEJ2000 (h m s) to all of the rows in the first column of bd
# if the row does not already have a value            
def addDEC(table):
    for i in range(table.num_rows):
        this_row = table.take(i)
        if this_row.column("DEJ2000")[0] == 'nan':
            dec = Simbad.query_object(this_row["Name"][0])
            dec = dec["DEC"]
            if dec[0][0] == '+':
                table[2][i] = dec[0][1:]
            else:
                table[2][i] = dec[0]


### Viewing Constraints for Brown Dwarf Candidates

Apart from the main database, you can also access a table of the viewing constraints for each of the targets. 

The 'observabilty_table.csv'currently contains the viewing constraints for the 92
objects in 'brownDwarfCandidates.csv'for an arbitrary date. The code below is used to update 'observability_table.csv' based on the information in 'brownDwarfCandidates.csv' and constraints defined by the user.

This requires the astroplan and astropy modules. 

<br>


In [ ]:
import numpy as np
from astroplan import (Observer, FixedTarget, is_observable, is_always_observable, months_observable, 
AltitudeConstraint, AirmassConstraint, AtNightConstraint)
from astropy.time import Time
from astropy.coordinates import SkyCoord
import astropy.units as u
from astroplan import download_IERS_A
from datascience import *
bd = Table().read_table('brownDwarfCandidates.csv')
#download_IERS_A()

*If there is an error, "WARNING: OldEarthOrientationDataWarning", then uncomment the 'download_IERS_A()' and re-run the above cell.



In [ ]:
target_table = bd.select("Name")
ra = make_array()
dec = make_array()
for i in range(bd.num_rows):
    coord = SkyCoord(bd["RAJ2000"][i] + " " + bd["DEJ2000"][i], unit=(u.hourangle, u.deg))
    ra = np.append(ra, coord.ra.deg)
    dec = np.append(dec, coord.dec.deg)
target_table.append_column("ra", ra)
target_table.append_column("dec", dec)
targets = []
for i in range(target_table.num_rows):
    ra = target_table[1][i]
    dec = target_table[2][i]
    name = target_table[0][i]
    targets.append(FixedTarget(coord=SkyCoord(ra=ra*u.deg, dec=dec*u.deg), name=name))

Change the constraints below as needed.

* Currently assumes observing from Palomar observatory on 8/1/2015 with airmass limit of 2

More information on setting various constraints can be found at https://astroplan.readthedocs.io/en/latest/tutorials/constraints.html

*The observability_table may take a while to load.


In [ ]:
palomar = Observer.at_site("Palomar")
time_range = Time(["2015-08-01 06:00", "2015-08-01 12:00"])
constraints = [AirmassConstraint(2), AtNightConstraint.twilight_civil()]

In [ ]:
ever_observable = is_observable(constraints, palomar, targets, time_range=time_range)
always_observable = is_always_observable(constraints, palomar, targets, time_range=time_range)
best_months = months_observable(constraints, palomar, targets)
observability_table = Table()
observability_table['targets'] = [target.name for target in targets]
observability_table['ever_observable'] = ever_observable
observability_table['always_observable'] = always_observable
observability_table['best_months'] = best_months
observability_table.to_df().to_csv('observability_table.csv', index = False)
observability_table